In [8]:
import pandas as pd #data processing, CSV file I/O (e.g. pd.read_csv) 
import numpy as np #linear algebra   

from sklearn import svm
from sklearn.metrics import roc_curve, auc
from sklearn import cross_validation
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier

#import accurecy_library for scoring attributes
from sklearn.metrics import accuracy_score
#importing the dataset
df = pd.read_csv('C:\\Users\\Ariful\\Desktop\\final data\\thesisfinaldata1.csv')

#filling null value as 0
df=df.fillna('0')

#Dropping Age column
df = df.drop(['Age'], axis= 1)
df = df.drop(['Age2'], axis= 1)
#Dropping Perm_Address column
df = df.drop(['Perm_Address'], axis= 1)

#taking independent variable
X = df.iloc[:,1:22].values
#taking dependent variable
y = df.iloc[:,-1].values

#print("\nX before making numerical: \n",X)
#print("\ny before making numerical: \n",y)

#taking careof categorical data
from sklearn.preprocessing import LabelEncoder
labelencoder_X = LabelEncoder()
for i in range(0,21):
    X[:,i]= labelencoder_X.fit_transform(X[:,i])

labelencoder_y = LabelEncoder()
y= labelencoder_y.fit_transform(y)
#print("\nX after making numerical: \n",X)
#print("\ny after making numerical: \n",y)


#df.head(5) # head method show only first 5 rows
for i in range(1,5):
    # split data train 90 % and test 10 %
    from sklearn.cross_validation import train_test_split
    #X_train=X,y_train=y
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.1, random_state = i)

    #import support vector machine classifier
    from sklearn.svm import SVC 
    #taking different peremater for support vector machine classifier
    svclassifier =OneVsRestClassifier(svm.SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,decision_function_shape=None, degree=3, gamma='auto', kernel='linear',max_iter=-1, probability=False, random_state=None, shrinking=True,tol=0.001, verbose=False))
    #fitting is equal to training. Then, after it is trained, the model can be used to make predictions, usually with a .predict() method call.
    y_score = svclassifier.fit(X_train, y_train).decision_function(X_test)
    
    #find out the predicted value
    predicted = svclassifier.predict(X_test) 

    #print("Prediction Result: ",predicted)

    #finding actual prediction compared with test size
    print("Accuracy: ",accuracy_score(y_test, predicted, normalize = True)) #accuracy_score(train output, predict output)

Accuracy:  0.5319148936170213
Accuracy:  0.6170212765957447
Accuracy:  0.6808510638297872
Accuracy:  0.5106382978723404


In [9]:
#import library for classification report, confusion matrix
from sklearn.metrics import classification_report, confusion_matrix  
#print(confusion_matrix(y_test,predicted))  
print(classification_report(y_test,predicted))



             precision    recall  f1-score   support

          0       0.00      0.00      0.00         1
          1       0.00      0.00      0.00         1
          2       0.41      0.82      0.55        11
          7       0.00      0.00      0.00         1
          9       0.00      0.00      0.00         1
         10       0.00      0.00      0.00         2
         11       0.00      0.00      0.00         1
         16       0.00      0.00      0.00         1
         17       0.00      0.00      0.00         1
         20       0.00      0.00      0.00         1
         24       0.00      0.00      0.00         2
         26       0.00      0.00      0.00         5
         27       0.60      0.79      0.68        19

avg / total       0.34      0.51      0.40        47



C:\Users\Ariful\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
# Applying k-Fold Cross Validation
from sklearn.model_selection import cross_val_score
#estimator = object, step = int or float, optional(default = 1), cv = int, cross-validation or an iterable optional
#scoring = string, callable or none, optional, default = none 
accuracies = cross_val_score(estimator = svclassifier, X = X_train, y = y_train, cv = 10)
#taking 10 cv result and find mean value
a=accuracies.mean()
print(a)
print(accuracies)
#accuracies.std()

C:\Users\Ariful\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


0.5986769365753194
[0.47619048 0.4893617  0.55813953 0.6        0.74358974 0.57894737
 0.67567568 0.59459459 0.59459459 0.67567568]


In [13]:
#printing predicted and actual predicted
print (predicted)
print (y_test)


[ 2  2 27  2  2  2 27 27 27 27 27 27 27 27 27  2  2 27  2 27 27 27  2  2
  2  2  2  2 27 27  2  2 27 27  2  2  2 27 27  2  2 27 27  2 27 27 27]
[20  2 27  7 26 11 27 24 27  2 27  0 24 27 17 27 26  2  2 10 27 27 10 27
  2  2 26  2 27 27  2  2 27 16  2 26  2 27 27 27  9 27 27 27  1 26 27]


In [16]:
n_classes = y.shape[-1]
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])


##############################################################################
# Plot of a ROC curve for a specific class
plt.figure()
plt.plot(fpr[2], tpr[2], label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

IndexError: too many indices for array

In [ ]:
# Applying Grid Search to find the best model and the best parameters
from sklearn.model_selection import GridSearchCV
#creat variable parameters where different perameters are taken
parameters = [{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
              {'C': [1, 10, 100, 1000], 'kernel': ['linear'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]}]
#creat variable grid_search where variable perameters are passed
#estimator = object, step = int or float, optional(default = 1), cv = int, cross-validation or an iterable optional
#scoring = string, callable or none, optional, default = none
grid_search = GridSearchCV(estimator = svclassifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)
#fitting is equal to training. Then, after it is trained, the model can be used to make predictions, usually with a .predict() method call.
grid_search = grid_search.fit(X_train, y_train)
#for best accurecy
best_accuracy = grid_search.best_score_
#storing grid_search.best_params__ in variable best_parameters
best_parameters = grid_search.best_params_
#print(grid_search)
print(best_accuracy)
print(best_parameters)